# SeqPandas Pileup Tutorial

This tutorial demonstrates how SeqPandas can create pileup representations from BAM/SAM files that replicate the output of the old deprecated `samtools mpileup` command, but with much faster performance due to the optimized `to_array` implementation.

In [1]:
import seqpandas as spd
from seqpandas.pileup import Pileup
from seqpandas.tools.pathing import pathing
import numpy as np
import pandas as pd

## What is a Pileup?

A pileup shows the alignment of reads at each position of a reference genome. It's essential for:
- Variant calling
- Coverage analysis
- Base quality assessment
- SNP/indel detection

The old `samtools mpileup` command was widely used but has been deprecated. SeqPandas provides a faster, more memory-efficient alternative.

## Sample Files

In [2]:
# Example files for demonstration
# Note: The pileup requires a single reference sequence
# We'll create a simple single-sequence FASTA for this demo
reference_fasta = "tests/test-data/example.fasta"
alignment_sam = "tests/test-data/example.sam"

# For this tutorial, we'll create a temporary single-sequence FASTA
# since the example.fasta has multiple sequences
import tempfile
from Bio import SeqIO

# Read the first sequence from the multi-sequence FASTA
with open(reference_fasta, "r") as f:
    first_seq = next(SeqIO.parse(f, "fasta"))

# Create a temporary single-sequence FASTA
with tempfile.NamedTemporaryFile(mode="w", suffix=".fasta", delete=False) as tmp_fasta:
    SeqIO.write([first_seq], tmp_fasta, "fasta")
    single_seq_fasta = tmp_fasta.name

print(f"Using single reference sequence: {first_seq.id}")
print(f"Sequence length: {len(first_seq.seq)} bp")

Using single reference sequence: seq1
Sequence length: 177 bp


## Basic Pileup Creation

The `Pileup` class creates a 2D matrix representation of aligned reads against a reference sequence.

**Important:** The Pileup class requires a single reference sequence in the FASTA file. If your FASTA contains multiple sequences, you'll need to extract the relevant one first (as we do in the cell above).

In [3]:
# Create a pileup object
# Note: Using the single-sequence FASTA we created above
pileup = Pileup(
    reference_file=single_seq_fasta,  # Using single-sequence file
    alignment_file=alignment_sam,
    minimum_coverage=5,  # Minimum coverage to call variants
    heterozygous_threshold=0.25,  # Threshold for heterozygous calls
    mimimum_alignment_coverage=0.75,  # Minimum fraction of alignment to include
)

=== Building Pileup From Scratch ===
=== Pilup Complete ===


## Understanding the Pileup Matrix Structure

The pileup matrix has 13 columns:
- Column 0: Variant call flag (1 if variant detected, 0 otherwise)
- Columns 1-4: Reference base counts (A, C, G, T)
- Columns 5-8: Alternative base counts (A, C, G, T)
- Columns 9-12: Insertion counts (A, C, G, T)

Each row represents a position in the reference sequence.

In [4]:
pileup.pileup

array([[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 2, 0, 0, 0, 1, 0, 3, 0],
       [0, 0, 0, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0],
       [0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
# View the first 10 positions of the pileup
print("Pileup matrix shape:", pileup.pileup.shape)
print("\nFirst 10 positions:")
print(pileup.pileup[:10])

Pileup matrix shape: (27, 13)

First 10 positions:
[[0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0 0 0 0]
 [0 2 0 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 2 0 0 0 0 0]
 [0 0 2 0 0 2 0 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0 2 0 0 0 0]
 [0 0 0 2 0 2 0 0 0 0 0 0 0]
 [0 0 2 0 0 2 0 0 0 1 0 3 0]
 [0 0 0 0 2 1 0 1 0 0 0 0 0]
 [0 0 3 0 0 2 0 0 1 0 0 0 0]]


In [6]:
# Create a more readable representation
columns = [
    "variant_flag",
    "ref_A",
    "ref_C",
    "ref_G",
    "ref_T",
    "alt_A",
    "alt_C",
    "alt_G",
    "alt_T",
    "ins_A",
    "ins_C",
    "ins_G",
    "ins_T",
]

pileup_df = pd.DataFrame(pileup.pileup[:20], columns=columns)
pileup_df.index.name = "position"
pileup_df.head(10)

,variant_flag,ref_A,ref_C,ref_G,ref_T,alt_A,alt_C,alt_G,alt_T,ins_A,ins_C,ins_G,ins_T
position,,,,,,,,,,,,,
0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,2,0,0,0,2,0,0,0,0,0,0,0
3,0,0,0,0,2,0,0,2,0,0,0,0,0
4,0,0,2,0,0,2,0,0,0,0,0,0,0
5,0,0,0,0,2,0,0,0,2,0,0,0,0
6,0,0,0,2,0,2,0,0,0,0,0,0,0
7,0,0,2,0,0,2,0,0,0,1,0,3,0
8,0,0,0,0,2,1,0,1,0,0,0,0,0


## Coverage Analysis

One of the most common uses of pileup data is coverage analysis.

In [7]:
# Calculate coverage at each position
coverage = pileup.pileup[:, 5:9].sum(axis=1)  # Sum alternative base counts

print(f"Mean coverage: {coverage.mean():.2f}")
print(f"Max coverage: {coverage.max()}")
print(f"Min coverage: {coverage.min()}")
print(f"Positions with coverage >= {pileup.minimum_coverage}: {(coverage >= pileup.minimum_coverage).sum()}")

Mean coverage: 1.26
Max coverage: 3
Min coverage: 0
Positions with coverage >= 5: 0


## Variant Detection

The pileup automatically detects potential variants based on the coverage and heterozygous thresholds.

In [8]:
# Find positions with detected variants
variant_positions = np.where(pileup.pileup[:, 0] == 1)[0]

print(f"Number of potential variants detected: {len(variant_positions)}")
if len(variant_positions) > 0:
    print(f"Variant positions: {variant_positions[:10]}...")  # Show first 10

    # Show details for the first variant
    first_variant_pos = variant_positions[0]
    print(f"\nDetails for variant at position {first_variant_pos}:")
    variant_data = pileup_df.iloc[first_variant_pos]
    print(variant_data)

Number of potential variants detected: 0


## Comparison with samtools mpileup

The traditional `samtools mpileup` command would output text like:
```
chr1  100  A  10  ..........  IIIIIIIIII
chr1  101  C  12  ....G.......  IIIIIIIIIIII
```

SeqPandas provides the same information in a structured numpy array that's:
- **Faster to process** - Optimized C/Cython implementation
- **More memory efficient** - Numpy arrays vs text parsing
- **Easier to analyze** - Direct array operations vs string parsing

In [9]:
# Simulate mpileup-style output from our pileup matrix
def format_as_mpileup(pileup_matrix, reference_seq, start_pos=0, end_pos=10):
    """Format pileup matrix as traditional mpileup output."""
    bases = ["A", "C", "G", "T"]

    for pos in range(start_pos, min(end_pos, len(pileup_matrix))):
        ref_base = reference_seq[pos] if pos < len(reference_seq) else "N"
        coverage = pileup_matrix[pos, 5:9].sum()

        # Build read base string
        read_bases = []
        for i, base in enumerate(bases):
            count = pileup_matrix[pos, 5 + i]  # Alt base counts
            if base == ref_base:
                read_bases.extend(["."] * count)  # Match
            else:
                read_bases.extend([base] * count)  # Mismatch

        read_base_str = "".join(read_bases) if read_bases else "*"
        print(f"ref\t{pos+1}\t{ref_base}\t{coverage}\t{read_base_str}")


# Display mpileup-style format
print("Position\tRef\tCov\tBases")
format_as_mpileup(pileup.pileup, pileup.ref_seq, 0, 10)

Position	Ref	Cov	Bases
ref	1	A	1	T
ref	2	T	1	.
ref	3	G	2	AA
ref	4	G	2	..
ref	5	A	2	..
ref	6	T	2	..
ref	7	T	2	AA
ref	8	T	2	AA
ref	9	A	2	.G
ref	10	T	3	AA.


## Performance Benefits

The key performance advantage comes from the optimized `to_array` function that uses Cython for fast array operations.

In [10]:
# Check if Cython extension is available
try:
    from seqpandas.cython_numpy import to_array

    print("✓ Using optimized Cython implementation")
    print("  This provides significant speedup for large BAM files")
except ImportError:
    print("✗ Cython extension not available")
    print("  Using pure Python fallback (slower but functional)")
    print("  To enable faster processing, rebuild with Cython support")

✓ Using optimized Cython implementation
  This provides significant speedup for large BAM files


## Saving and Loading Pileups

For large files, you can save the computed pileup to avoid recomputation.

In [11]:
# Save pileup for later use
output_file = "example_pileup.npy"
pileup.save_pileup(output_file)
print(f"Pileup saved to {output_file}")

# Load a previously saved pileup
# This is much faster than recomputing from BAM/SAM
pileup_fast = Pileup(
    reference_file=single_seq_fasta,  # Using single-sequence file
    alignment_file=alignment_sam,
    use_saved_pileup=output_file,  # Load from saved file
)
print("Pileup loaded from saved file")

Pileup saved to example_pileup.npy
=== Using Input Pileup ===
=== Pilup Complete ===
Pileup loaded from saved file


## Advanced Usage: Filtering and Quality Control

In [12]:
# Access mapping coverage statistics
if pileup.mapping_coverages:
    mapping_stats = pd.Series(pileup.mapping_coverages)
    print("Alignment quality statistics:")
    print(f"  Mean mapping coverage: {mapping_stats.mean():.3f}")
    print(f"  Min mapping coverage: {mapping_stats.min():.3f}")
    print(f"  Max mapping coverage: {mapping_stats.max():.3f}")
    print(f"  Alignments passing filter: {len(mapping_stats)}")

Alignment quality statistics:
  Mean mapping coverage: 0.953
  Min mapping coverage: 0.812
  Max mapping coverage: 1.000
  Alignments passing filter: 4


## Working with Real BAM Files

For production use with real BAM files, you would typically:

```python
# Example with real data (not run here due to file size)
pileup = Pileup(
    reference_file='reference_genome.fasta',
    alignment_file='sample.bam',
    minimum_coverage=30,  # Higher for real data
    heterozygous_threshold=0.25,
    mimimum_alignment_coverage=0.90,  # Stricter for real data
    save_pileup_to_destination='sample_pileup.npy'  # Save for reuse
)
```

## Integration with Machine Learning

The numpy array format makes it easy to use pileup data as input for machine learning models.

In [14]:
# Example: Prepare pileup data for ML model
# Extract features for positions with sufficient coverage
min_coverage_for_ml = 10
coverage = pileup.pileup[:, 5:9].sum(axis=1)
high_coverage_mask = coverage >= min_coverage_for_ml

# Feature matrix: base counts normalized by coverage
features = pileup.pileup[high_coverage_mask, 5:9]  # Alt base counts
coverage_filtered = coverage[high_coverage_mask, np.newaxis]
normalized_features = features / (coverage_filtered + 1e-6)  # Avoid division by zero

# Labels: variant flags
labels = pileup.pileup[high_coverage_mask, 0]

print(f"ML-ready feature matrix shape: {normalized_features.shape}")
print(f"Number of positive examples (variants): {labels.sum():.0f}")
print(f"Number of negative examples (reference): {(1-labels).sum():.0f}")

ML-ready feature matrix shape: (0, 4)
Number of positive examples (variants): 0
Number of negative examples (reference): 0


## Summary

SeqPandas Pileup provides:
- **Fast conversion** of BAM/SAM alignments to numpy arrays
- **Automatic variant detection** based on configurable thresholds
- **Memory-efficient storage** using numpy arrays
- **Easy integration** with pandas DataFrames and ML frameworks
- **Significant speedup** over text-based mpileup parsing

The structured array format makes downstream analysis much simpler than parsing traditional mpileup text output.